In [1]:
import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras.layers import Input, InputLayer, Conv2D, Dense, Lambda, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import Callback, EarlyStopping
from sklearn.model_selection import train_test_split
import time

from tensorflow_privacy.privacy.analysis.rdp_accountant import compute_rdp
from tensorflow_privacy.privacy.analysis.rdp_accountant import get_privacy_spent
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasAdamOptimizer
from tensorflow_privacy.privacy.analysis.compute_noise_from_budget_lib import compute_noise
from tensorflow_privacy.privacy.keras_models.dp_keras_model import DPSequential


from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack import membership_inference_attack as mia
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import AttackInputData
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import SlicingSpec
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import AttackType

import tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.plotting as plotting

import numpy as np
from scipy import special

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
learning_rate = 15e-5
batch_size=100

In [4]:
def membership_inference_attack(model, X_train, X_test, y_train, y_test):
    print('Predict on train...')
    logits_train = model.predict(X_train, batch_size=batch_size)
    print('Predict on test...')
    logits_test = model.predict(X_test, batch_size=batch_size)

    print('Apply softmax to get probabilities from logits...')
    prob_train = special.softmax(logits_train, axis=1)
    prob_test = special.softmax(logits_test, axis=1)

    print('Compute losses...')
    cce = tf.keras.backend.categorical_crossentropy
    constant = tf.keras.backend.constant

    loss_train = cce(constant(y_train), constant(prob_train), from_logits=False).numpy()
    loss_test = cce(constant(y_test), constant(prob_test), from_logits=False).numpy()
    
    labels_train = np.argmax(y_train, axis=1)
    labels_test = np.argmax(y_test, axis=1)

    input = AttackInputData(
      logits_train = logits_train,
      logits_test = logits_test,
      loss_train = loss_train,
      loss_test = loss_test,
      labels_train = labels_train,
      labels_test = labels_test
    )

    # Run several attacks for different data slices
    attacks_result = mia.run_attacks(input,
                                     SlicingSpec(
                                         entire_dataset = True,
                                         by_class = True,
                                         by_classification_correctness = True
                                     ),
                                     attack_types = [
                                         AttackType.THRESHOLD_ATTACK,
                                         AttackType.LOGISTIC_REGRESSION,
                                         AttackType.MULTI_LAYERED_PERCEPTRON,
                                         AttackType.RANDOM_FOREST, 
                                         AttackType.K_NEAREST_NEIGHBORS,
                                         AttackType.THRESHOLD_ENTROPY_ATTACK
                                     ])

    # Plot the ROC curve of the best classifier
#     fig = plotting.plot_roc_curve(
#         attacks_result.get_result_with_max_auc().roc_curve)

    # Print a user-friendly summary of the attacks
    print(attacks_result.summary(by_slices = True))
    time.sleep(5)
    return attacks_result.get_result_with_max_auc().get_auc(), attacks_result.get_result_with_max_attacker_advantage().get_attacker_advantage()

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [5]:
def feature_extractor():
    input_tensor = Input(shape=(32, 32, 3))
    resized_images = Lambda(lambda image: tf.image.resize(image, (224, 224)))(input_tensor)
    base_model = applications.DenseNet201(include_top=False,
                                          weights='imagenet',
                                          input_tensor=resized_images,
                                          input_shape=(224, 224, 3),
                                          pooling='max')
    
    output = base_model.layers[-1].output
    base_model = Model(inputs=input_tensor, outputs=output)
    return base_model

In [6]:
# Load in the data
cifar10 = tf.keras.datasets.cifar10
cifar10_categories = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
(X_train, y_train_), (X_test, y_test_) = cifar10.load_data()
X_train_flat, X_test_flat = X_train.reshape(-1, 32*32*3), X_test.reshape(-1, 32*32*3)
X_train, X_test = X_train / 255.0, X_test / 255.0
y_train, y_test = np.eye(10)[y_train_.flatten()], np.eye(10)[y_test_.flatten()]

In [7]:
feature_extractor = feature_extractor()

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
train_generator = train_datagen.flow(X_train,
                                     y_train,
                                     batch_size=batch_size,
                                     shuffle=False)
features_train = feature_extractor.predict(train_generator)

# repeat the same operation with the test data (here used for validation)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
val_generator = val_datagen.flow(X_test,
                                 y_test,
                                 batch_size=batch_size,
                                 shuffle=False)
features_valid = feature_extractor.predict(val_generator)

In [8]:
def create_network(dropout=None, regularizer=None):
    input_data = Input(shape=features_train.shape[1])
    if dropout is not None:
        x = Dropout(dropout)(input_data)
        x = Dense(256, activation='relu', kernel_regularizer=regularizer)(x)
    else:
        x = Dense(256, activation='relu', kernel_regularizer=regularizer)(input_data)
    if dropout is not None:
        x = Dropout(dropout)(x)
    output = Dense(10, kernel_regularizer=regularizer)(x)
        
    model = Model(input_data, output)
    
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
            
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=loss,
                  metrics=['accuracy'])
    return model

def create_networkDP(noise_multiplier, l2_norm_clip, microbatches):
    model = Sequential(
        layers = [
            Input(shape=features_train.shape[1]),
            Dense(256, 
                  activation='relu',
                  kernel_initializer=tf.keras.initializers.he_normal()),
            Dense(10,
                  kernel_initializer=tf.keras.initializers.he_normal())
        ]
    )

    optimizer = DPKerasAdamOptimizer(
                            l2_norm_clip=l2_norm_clip,
                            noise_multiplier=noise_multiplier,
                            num_microbatches=microbatches,
                            learning_rate=learning_rate)
    
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True, reduction=tf.losses.Reduction.NONE)
    
    model.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy'])
    
    return model

In [11]:
epochs = 50
batch_size = 100
attacks = 1
settings = [
    (None,None),
    (0.25,None),
    (0.50,None),
    (0.75,None),
    (None,'l2'),
    (0.25,'l2'),
    (0.50,'l2'),
    (0.75,'l2'),
]
results_summary = []

for drop, reg in settings: 
    # Instantiate network
    classifier = create_network(dropout=drop, regularizer=reg)
    
    # Train network until convergence    
    start_time = time.time()
    r = classifier.fit(features_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(features_valid, y_test),
                    shuffle=True,
                    verbose=1
                   )
    end_time = time.time()
    model = Sequential([feature_extractor, classifier])
    model.compile(loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True))
    time_elapsed = (end_time - start_time)
    
    # MIA
    aauc = []
    aadv = []
    for _ in range(attacks):
        auc, adv = membership_inference_attack(model, X_train, X_test, y_train, y_test)
        aauc.append(auc)
        aadv.append(adv)
    mauc = sum(aauc) / attacks
    madv = sum(aadv) / attacks

    # Write result summary
    summ = ', '.join(map(str,[
        len(r.history['loss']), #epochs
        drop,
        reg,
        r.history['loss'][-1], 
        r.history['val_loss'][-1],
        r.history['accuracy'][-1],
        r.history['val_accuracy'][-1],
        time_elapsed,
        mauc,
        madv
    ]))

    results_summary.append(summ)
    print('='*40)
    
print('Epochs, Dropout, Regularizer, Loss, Val loss, Accuracy, Val accuracy, Time, AUC, Advantage')
for r in results_summary:
    print(r)

Epoch 1/50
500/500 [==============================] - 1s 2ms/step - loss: 0.8307 - accuracy: 0.7200 - val_loss: 0.5978 - val_accuracy: 0.7951
Epoch 2/50
500/500 [==============================] - 1s 2ms/step - loss: 0.5120 - accuracy: 0.8235 - val_loss: 0.5215 - val_accuracy: 0.8192
Epoch 3/50
500/500 [==============================] - 1s 2ms/step - loss: 0.4443 - accuracy: 0.8461 - val_loss: 0.5012 - val_accuracy: 0.8295
Epoch 4/50
500/500 [==============================] - 1s 2ms/step - loss: 0.4012 - accuracy: 0.8616 - val_loss: 0.4755 - val_accuracy: 0.8404
Epoch 5/50
500/500 [==============================] - 1s 2ms/step - loss: 0.3708 - accuracy: 0.8726 - val_loss: 0.4636 - val_accuracy: 0.8443
Epoch 6/50
500/500 [==============================] - 1s 2ms/step - loss: 0.3438 - accuracy: 0.8813 - val_loss: 0.4622 - val_accuracy: 0.8421
Epoch 7/50
500/500 [==============================] - 1s 2ms/step - loss: 0.3202 - accuracy: 0.8892 - val_loss: 0.4479 - val_accuracy: 0.8511
Epoch 

C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario

Best-performing attacks over all slices
  RANDOM_FOREST (with 1000 training and 1000 test examples) achieved an AUC of 0.78 on slice CLASS=3
  RANDOM_FOREST (with 1000 training and 1000 test examples) achieved an advantage of 0.47 on slice CLASS=3

Best-performing attacks over slice: "Entire dataset"
  RANDOM_FOREST (with 10000 training and 10000 test examples) achieved an AUC of 0.69
  RANDOM_FOREST (with 10000 training and 10000 test examples) achieved an advantage of 0.27

Best-performing attacks over slice: "CLASS=0"
  RANDOM_FOREST (with 1000 training and 1000 test examples) achieved an AUC of 0.67
  RANDOM_FOREST (with 1000 training and 1000 test examples) achieved an advantage of 0.26

Best-performing attacks over slice: "CLASS=1"
  RANDOM_FOREST (with 1000 training and 1000 test examples) achieved an AUC of 0.63
  RANDOM_FOREST (with 1000 training and 1000 test examples) achieved an advantage of 0.22

Best-performing attacks over slice: "CLASS=2"
  LOGISTIC_REGRESSION (with 100

500/500 [==============================] - 1s 2ms/step - loss: 0.3765 - accuracy: 0.8662 - val_loss: 0.4126 - val_accuracy: 0.8588
Epoch 38/50
500/500 [==============================] - 1s 2ms/step - loss: 0.3696 - accuracy: 0.8678 - val_loss: 0.4050 - val_accuracy: 0.8617
Epoch 39/50
500/500 [==============================] - 1s 2ms/step - loss: 0.3668 - accuracy: 0.8686 - val_loss: 0.4027 - val_accuracy: 0.8619
Epoch 40/50
500/500 [==============================] - 1s 2ms/step - loss: 0.3575 - accuracy: 0.8738 - val_loss: 0.4033 - val_accuracy: 0.8616
Epoch 41/50
500/500 [==============================] - 1s 2ms/step - loss: 0.3610 - accuracy: 0.8728 - val_loss: 0.4118 - val_accuracy: 0.8598
Epoch 42/50
500/500 [==============================] - 1s 2ms/step - loss: 0.3647 - accuracy: 0.8682 - val_loss: 0.4085 - val_accuracy: 0.8625
Epoch 43/50
500/500 [==============================] - 1s 2ms/step - loss: 0.3563 - accuracy: 0.8737 - val_loss: 0.4048 - val_accuracy: 0.8600
Epoch 44/50

C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario

Best-performing attacks over all slices
  RANDOM_FOREST (with 1373 training and 1373 test examples) achieved an AUC of 0.69 on slice CORRECTLY_CLASSIFIED=False
  RANDOM_FOREST (with 1373 training and 1373 test examples) achieved an advantage of 0.31 on slice CORRECTLY_CLASSIFIED=False

Best-performing attacks over slice: "Entire dataset"
  MULTI_LAYERED_PERCEPTRON (with 10000 training and 10000 test examples) achieved an AUC of 0.57
  LOGISTIC_REGRESSION (with 10000 training and 10000 test examples) achieved an advantage of 0.13

Best-performing attacks over slice: "CLASS=0"
  LOGISTIC_REGRESSION (with 1000 training and 1000 test examples) achieved an AUC of 0.59
  LOGISTIC_REGRESSION (with 1000 training and 1000 test examples) achieved an advantage of 0.16

Best-performing attacks over slice: "CLASS=1"
  RANDOM_FOREST (with 1000 training and 1000 test examples) achieved an AUC of 0.55
  RANDOM_FOREST (with 1000 training and 1000 test examples) achieved an advantage of 0.12

Best-perfo

Epoch 35/50
500/500 [==============================] - 1s 2ms/step - loss: 0.6842 - accuracy: 0.7628 - val_loss: 0.5269 - val_accuracy: 0.8350
Epoch 36/50
500/500 [==============================] - 1s 2ms/step - loss: 0.6801 - accuracy: 0.7645 - val_loss: 0.5260 - val_accuracy: 0.8377
Epoch 37/50
500/500 [==============================] - 1s 2ms/step - loss: 0.6830 - accuracy: 0.7635 - val_loss: 0.5145 - val_accuracy: 0.8403
Epoch 38/50
500/500 [==============================] - 1s 2ms/step - loss: 0.6785 - accuracy: 0.7667 - val_loss: 0.5123 - val_accuracy: 0.8376
Epoch 39/50
500/500 [==============================] - 1s 2ms/step - loss: 0.6808 - accuracy: 0.7662 - val_loss: 0.5138 - val_accuracy: 0.8415
Epoch 40/50
500/500 [==============================] - 1s 2ms/step - loss: 0.6712 - accuracy: 0.7676 - val_loss: 0.5105 - val_accuracy: 0.8376
Epoch 41/50
500/500 [==============================] - 1s 2ms/step - loss: 0.6714 - accuracy: 0.7667 - val_loss: 0.5037 - val_accuracy: 0.8414

C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario

KeyboardInterrupt: 

In [10]:
# TEST FOR DIFFERENT EPS
results_summary = []

n = X_train.shape[0]
epochs = 50
batch_size = 100
microbatches = 100
epsilons = [0.1,0.5,1,2,4,8,16,100,1000]
delta = 1e-6
min_noise = 1e-100
l2_norm_clip = 1
sampling_rate = batch_size / n
attacks = 1

for e in epsilons:
    # Compute noise multiplier from target epsilon
    noise_multiplier = compute_noise(n, batch_size, e, epochs, delta, min_noise)
    
    # Compute epsilon
    orders = [1 + x / 10. for x in range(1, 100)] + list(range(11, 101))
    sampling_probability = batch_size / n
    rdp = compute_rdp(q=sampling_probability,
                    noise_multiplier=noise_multiplier,
                    steps=epochs * n // batch_size,
                    orders=orders)
    eps = get_privacy_spent(orders, rdp, target_delta=delta)

    # Instantiate network
    classifierDP = create_networkDP(noise_multiplier, l2_norm_clip, microbatches)
    
    # Train network until convergence    
    start_time = time.time()
    r = classifierDP.fit(features_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(features_valid, y_test),
                    shuffle=True,
                    #callbacks=[lr_reduce],
                    verbose=1
                   )
    end_time = time.time()

    modelDP = Sequential([feature_extractor, classifierDP])
    modelDP.compile(loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True))
    time_elapsed = (end_time - start_time)

    # MIA 
    aauc = []
    aadv = []
    for _ in range(attacks):
        auc, adv = membership_inference_attack(modelDP, X_train, X_test, y_train, y_test)
        aauc.append(auc)
        aadv.append(adv)
    mauc = sum(aauc) / attacks
    madv = sum(aadv) / attacks

    # Write result summary
    summ = ', '.join(map(str,[
          len(r.history['loss']),
          e,
          delta,
          l2_norm_clip,
          noise_multiplier,
          sampling_rate,
          eps[0],
          r.history['loss'][-1], 
          r.history['val_loss'][-1],
          r.history['accuracy'][-1],
          r.history['val_accuracy'][-1],
          time_elapsed,
          mauc,
          madv
    ]))
    results_summary.append(summ)
    print('='*40)

print('Epochs, Target epsilon, delta, C, Sigma, Sampling rate, Epsilon, Loss, Val loss, Accuracy, Val accuracy, Time, AUC, Advantage')    
for r in results_summary:
    print(r)

DP-SGD with sampling rate = 0.2% and noise_multiplier = 13.133789235896893 iterated over 25000 steps satisfies differential privacy with eps = 0.1 and delta = 1e-06.
Epoch 1/50
500/500 [==============================] - 34s 66ms/step - loss: 4.1917 - accuracy: 0.1249 - val_loss: 3.2711 - val_accuracy: 0.1460
Epoch 2/50
500/500 [==============================] - 32s 64ms/step - loss: 3.1037 - accuracy: 0.1560 - val_loss: 2.9478 - val_accuracy: 0.1727
Epoch 3/50
500/500 [==============================] - 32s 63ms/step - loss: 2.8494 - accuracy: 0.1827 - val_loss: 2.7261 - val_accuracy: 0.2022
Epoch 4/50
500/500 [==============================] - 32s 64ms/step - loss: 2.6689 - accuracy: 0.2100 - val_loss: 2.5857 - val_accuracy: 0.2309
Epoch 5/50
500/500 [==============================] - 33s 66ms/step - loss: 2.5520 - accuracy: 0.2382 - val_loss: 2.4776 - val_accuracy: 0.2561
Epoch 6/50
500/500 [==============================] - 32s 64ms/step - loss: 2.4293 - accuracy: 0.2672 - val_loss: 

C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario

Best-performing attacks over all slices
  MULTI_LAYERED_PERCEPTRON (with 1000 training and 1000 test examples) achieved an AUC of 0.56 on slice CLASS=0
  MULTI_LAYERED_PERCEPTRON (with 1000 training and 1000 test examples) achieved an advantage of 0.11 on slice CLASS=0

Best-performing attacks over slice: "Entire dataset"
  MULTI_LAYERED_PERCEPTRON (with 10000 training and 10000 test examples) achieved an AUC of 0.50
  LOGISTIC_REGRESSION (with 10000 training and 10000 test examples) achieved an advantage of 0.02

Best-performing attacks over slice: "CLASS=0"
  MULTI_LAYERED_PERCEPTRON (with 1000 training and 1000 test examples) achieved an AUC of 0.56
  MULTI_LAYERED_PERCEPTRON (with 1000 training and 1000 test examples) achieved an advantage of 0.11

Best-performing attacks over slice: "CLASS=1"
  THRESHOLD_ENTROPY_ATTACK (with 5000 training and 1000 test examples) achieved an AUC of 0.51
  MULTI_LAYERED_PERCEPTRON (with 1000 training and 1000 test examples) achieved an advantage of 

Epoch 32/50
500/500 [==============================] - 33s 65ms/step - loss: 0.9501 - accuracy: 0.7537 - val_loss: 1.0176 - val_accuracy: 0.7423
Epoch 33/50
500/500 [==============================] - 32s 64ms/step - loss: 0.9498 - accuracy: 0.7556 - val_loss: 1.0200 - val_accuracy: 0.7432
Epoch 34/50
500/500 [==============================] - 33s 65ms/step - loss: 0.9514 - accuracy: 0.7567 - val_loss: 1.0179 - val_accuracy: 0.7439
Epoch 35/50
500/500 [==============================] - 33s 65ms/step - loss: 0.9449 - accuracy: 0.7585 - val_loss: 1.0167 - val_accuracy: 0.7457
Epoch 36/50
500/500 [==============================] - 32s 65ms/step - loss: 0.9488 - accuracy: 0.7584 - val_loss: 1.0215 - val_accuracy: 0.7481
Epoch 37/50
500/500 [==============================] - 32s 64ms/step - loss: 0.9500 - accuracy: 0.7604 - val_loss: 1.0203 - val_accuracy: 0.7483
Epoch 38/50
500/500 [==============================] - 32s 65ms/step - loss: 0.9514 - accuracy: 0.7617 - val_loss: 1.0199 - val_ac

C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario

...

C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario

Best-performing attacks over all slices
  MULTI_LAYERED_PERCEPTRON (with 1000 training and 1000 test examples) achieved an AUC of 0.58 on slice CLASS=5
  MULTI_LAYERED_PERCEPTRON (with 1000 training and 1000 test examples) achieved an advantage of 0.14 on slice CLASS=7

Best-performing attacks over slice: "Entire dataset"
  THRESHOLD_ATTACK (with 50000 training and 10000 test examples) achieved an AUC of 0.51
  THRESHOLD_ENTROPY_ATTACK (with 50000 training and 10000 test examples) achieved an advantage of 0.03

Best-performing attacks over slice: "CLASS=0"
  K_NEAREST_NEIGHBORS (with 1000 training and 1000 test examples) achieved an AUC of 0.54
  MULTI_LAYERED_PERCEPTRON (with 1000 training and 1000 test examples) achieved an advantage of 0.08

Best-performing attacks over slice: "CLASS=1"
  RANDOM_FOREST (with 1000 training and 1000 test examples) achieved an AUC of 0.55
  RANDOM_FOREST (with 1000 training and 1000 test examples) achieved an advantage of 0.10

Best-performing attacks 

500/500 [==============================] - 32s 64ms/step - loss: 0.7102 - accuracy: 0.8506 - val_loss: 0.8662 - val_accuracy: 0.8277
Epoch 33/50
500/500 [==============================] - 32s 64ms/step - loss: 0.7116 - accuracy: 0.8504 - val_loss: 0.8615 - val_accuracy: 0.8300
Epoch 34/50
500/500 [==============================] - 32s 64ms/step - loss: 0.7070 - accuracy: 0.8517 - val_loss: 0.8599 - val_accuracy: 0.8298
Epoch 35/50
500/500 [==============================] - 33s 65ms/step - loss: 0.7043 - accuracy: 0.8522 - val_loss: 0.8558 - val_accuracy: 0.8309
Epoch 36/50
500/500 [==============================] - 32s 64ms/step - loss: 0.7045 - accuracy: 0.8541 - val_loss: 0.8613 - val_accuracy: 0.8316
Epoch 37/50
500/500 [==============================] - 32s 65ms/step - loss: 0.7015 - accuracy: 0.8540 - val_loss: 0.8575 - val_accuracy: 0.8306
Epoch 38/50
500/500 [==============================] - 32s 64ms/step - loss: 0.7003 - accuracy: 0.8543 - val_loss: 0.8640 - val_accuracy: 0.83

C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario

Best-performing attacks over all slices
  K_NEAREST_NEIGHBORS (with 1000 training and 1000 test examples) achieved an AUC of 0.55 on slice CLASS=9
  MULTI_LAYERED_PERCEPTRON (with 1000 training and 1000 test examples) achieved an advantage of 0.13 on slice CLASS=6

Best-performing attacks over slice: "Entire dataset"
  RANDOM_FOREST (with 10000 training and 10000 test examples) achieved an AUC of 0.52
  RANDOM_FOREST (with 10000 training and 10000 test examples) achieved an advantage of 0.04

Best-performing attacks over slice: "CLASS=0"
  K_NEAREST_NEIGHBORS (with 1000 training and 1000 test examples) achieved an AUC of 0.53
  MULTI_LAYERED_PERCEPTRON (with 1000 training and 1000 test examples) achieved an advantage of 0.10

Best-performing attacks over slice: "CLASS=1"
  RANDOM_FOREST (with 1000 training and 1000 test examples) achieved an AUC of 0.54
  RANDOM_FOREST (with 1000 training and 1000 test examples) achieved an advantage of 0.12

Best-performing attacks over slice: "CLASS=